<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/langchain_refined_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install  accelerate -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [ ]:
from torch import cuda, bfloat16
import transformers

model_id="Trelis/Llama-2-7b-chat-hf-sharded-bf16"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'HF_AUTH_TOKEN'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded on cuda:0


In [39]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [40]:
! pip install -qq -U langchain==0.0.262

In [41]:
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained(model_id)

max_len = 2048 # 1024
task = "text-generation"
T = 0

pipe = pipeline(
    task=task,
    model=model,
    tokenizer=tokenizer,
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15,
    pad_token_id = 11
)

llm = HuggingFacePipeline(pipeline=pipe)

In [42]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine")


In [ ]:
!pip install youtube-transcript-api -qq


In [ ]:
!pip install pytube -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 872.0 kB/s eta 0:00:00


In [44]:
from langchain.document_loaders import YoutubeLoader

# Use the YoutubeLoader to load and parse the transcript of a YouTube video
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=O5nskjZ_GoI", add_video_info=True)
video = loader.load()

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=50)


In [46]:
texts = text_splitter.split_documents(video)

In [55]:
from langchain import PromptTemplate

question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [51]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [52]:
refine_outputs = refine_chain({"input_documents": texts})


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [53]:
refine_outputs

{'input_documents': [Document(page_content='Hello world, I’m Carrie Anne, and welcome\nto CrashCourse Computer Science! Over the course of this series, we’re going\nto go from bits, bytes, transistors and logic gates, all the way to Operating Systems, Virtual\nReality and Robots! We’re going to cover a lot, but just to\nclear things up - we ARE NOT going to teach you how to program. Instead, we’re going to explore a range\nof computing topics as a discipline and a technology. Computers are the lifeblood of today’s world. If they were to suddenly turn off, all at\nonce, the power grid would shut down, cars would crash, planes would fall, water treatment\nplants would stop, stock markets would freeze, trucks with food wouldn’t know where to\ndeliver, and employees wouldn’t get paid. Even many non-computer objects - like DFTBA\nshirts and the chair I’m sitting on – are made in factories run by computers. Computing really has transformed nearly every\naspect of our lives. And this isn’t th

In [54]:
refine_outputs['output_text']

' • Hollerith developed a machine that could process data using punch cards and electrical circuits.\n               • His machine was 10 times faster than manual tabulation methods and saved the Census Office millions of dollars.\n               • Businesses recognized the value of computing and saw its potential to boost profits.\n               • Hollerith founded The Tabulating Machine Company, which later became IBM.\n               • Electro-mechanical "business machines" transformed commerce and government.\n               • The explosion in world population and globalized trade led to the need for faster and more flexible data processing tools.'